In [2]:
from Bio import SeqIO
import re

In [25]:
f = open("../OUTPUTS/antismash_finals/T_712_11/mg.assembly.merged.length_filtered/knownclusterblastoutput.txt", 'r')
#for Line in f:
 #       columns = Line.split('|')
  #      BGC_id = columns[5].split('\t')[1]
   #     print(BGC_id)
    

In [34]:
def get_cds_feature_with_qualifier_value(seq_record, name, value):
    """Function to look for CDS feature by annotation value in sequence record.
    
    e.g. You can use this for finding features by locus tag, gene ID, or protein ID.
    """
    # Loop over the features
    for feature in genome_record.features:
        if feature.type == "CDS" and value in feature.qualifiers.get(name, []):
            return feature
    # Could not find it
    return None

genome_record = SeqIO.read("../OUTPUTS/antismash_finals/T_712_11/mg.assembly.merged.length_filtered/test_contig_318127.region001.gbk", "genbank")
cds_feature = get_cds_feature_with_qualifier_value(genome_record, "locus_tag", "JJJEIFBD_433319")

print(cds_feature)

type: CDS
location: [19774:20053](+)
qualifiers:
    Key: ID, Value: ['JJJEIFBD_433319']
    Key: gene, Value: ['JJJEIFBD_433319']
    Key: inference, Value: ['ab initio prediction:Prodigal_v2.6.3']
    Key: locus_tag, Value: ['JJJEIFBD_433319']
    Key: partial, Value: ['00']
    Key: phase, Value: ['0']
    Key: product, Value: ['unannotated protein']
    Key: source, Value: ['Prodigal_v2.6.3']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ['MNAINLPDSASDTASNADPTSDVRLRVVESRYRVEYAQLVEGATILAFERDLESDDVELCRSRLAQLQRTHGADKVTVVIWDTTDERIVFAG']



In [36]:

lines = []
line = ''
for i in range(len(genome_record.features)):
    #check if the type of region/feature is a coding sequence/annotated region with the 'phase' qualifier/key, to also include along the line
    if genome_record.features[i].qualifiers.__contains__('phase'):
        #check if annotation is from a 'tool' a 'aStool' or a 'source' to include in the second column 
        if genome_record.features[i].qualifiers.__contains__('tool'):
            string = str(genome_record.description) + '\t' + str(genome_record.features[i].qualifiers['tool'][0]) + ' ' + str(genome_record.features[i].type) + '\t' + str(genome_record.features[i].location.start) + '\t' + str(genome_record.features[i].location.end) + '\t' + '.' + '\t' + str(genome_record.features[i].location)[-2] + '\t' + str(genome_record.features[i].qualifiers['phase'][0]) + '\t'
        else:
            if genome_record.features[i].qualifiers.__contains__('aStool'):
                string = str(genome_record.description) + '\t' + str(genome_record.features[i].qualifiers['aStool'][0]) + ' ' + str(genome_record.features[i].type) + '\t' + str(genome_record.features[i].location.start) + '\t' + str(genome_record.features[i].location.end) + '\t' + '.' + '\t' + str(genome_record.features[i].location)[-2] + '\t' + str(genome_record.features[i].qualifiers['phase'][0]) + '\t'
            else:
                string = str(genome_record.description) + '\t' + str(genome_record.features[i].qualifiers['source'][0]) + ' ' + str(genome_record.features[i].type) + '\t' + str(genome_record.features[i].location.start) + '\t' + str(genome_record.features[i].location.end) + '\t' + '.' + '\t' + str(genome_record.features[i].location)[-2] + '\t' + str(genome_record.features[i].qualifiers['phase'][0]) + '\t'
    else:
        if genome_record.features[i].qualifiers.__contains__('tool'):
            string = str(genome_record.description) + '\t' + str(genome_record.features[i].qualifiers['tool'][0]) + ' ' + str(genome_record.features[i].type) + '\t' + str(genome_record.features[i].location.start) + '\t' + str(genome_record.features[i].location.end) + '\t' + '.' + '\t' + str(genome_record.features[i].location)[-2] + '\t' 
        else:
            if genome_record.features[i].qualifiers.__contains__('aStool'):
                string = str(genome_record.description) + '\t' + str(genome_record.features[i].qualifiers['aStool'][0]) + ' ' + str(genome_record.features[i].type) + '\t' + str(genome_record.features[i].location.start) + '\t' + str(genome_record.features[i].location.end) + '\t' + '.' + '\t' + str(genome_record.features[i].location)[-2] + '\t'
            else:
                string = str(genome_record.description) + '\t' + str(genome_record.features[i].qualifiers['source'][0]) + ' ' + str(genome_record.features[i].type) + '\t' + str(genome_record.features[i].location.start) + '\t' + str(genome_record.features[i].location.end) + '\t' + '.' + '\t' + str(genome_record.features[i].location)[-2] + '\t'

    attributess = ''
    for key in genome_record.features[i].qualifiers:
    #print(i, qualif_dict[i])
        if key ==  'ID' or key == 'gene' or key == 'translation' or key == 'partial' or key == 'domain_id':
            pass
        else:
            if len(genome_record.features[i].qualifiers[key]) > 1:
                temp = str(key) + '=' + str(genome_record.features[i].qualifiers[key]) + ';' 
                attributess = attributess + str(temp)
            else:
            #print(len(genome_record.features[i].qualifiers[key]))
                temp = str(key) + '=' + str(genome_record.features[i].qualifiers[key][0]) + ';' 
                attributess = attributess + str(temp)
    
    #attributess 
    line = line + string + attributess
    lines.append(str(line))
    line = ''
    #print(line)
#print(line)
#lines.append(str(line))
#line = ''
#print(lines)

In [ ]:


#function to get the highest scoring BGC term from the knownclusterblast text file for each of the found loci
def get_bgc_terms(file, list):
    #open file
    f = open(file[0], 'r')


    #dictionary to store the MiBig identifier with the corresponding locus_tag 
    locus_dict = {}    

    for Line in f:
        columns = Line.split('|')
        locus = columns[4]
        BGC_id = columns[5].split('\t')[1]
        #The highest is the first so if the key is already in the dictionary skip so no others are added for that locus tag
        if locus in locus_dict.keys():
            pass
        else:
            locus_dict[locus] = BGC_id
    
    file_list = list 
    for key, value in locus_dict.items():
        for line in file_list:
            if key in line:
                #print(line)
                line = line + value 

    f.close()
    return file_list

    

final_list = get_bgc_terms(cb_txtfile, file_list)


In [55]:
f = open("../OUTPUTS/antismash_finals/T_712_11/mg.assembly.merged.length_filtered/knownclusterblastoutput.txt", 'r')


#dictionary to store the MiBig identifier with the corresponding locus_tag 
locus_dict = {}    

for Line in f:
    columns = Line.split('|')
    locus = columns[4]
    BGC_id = columns[5].split('\t')[1]
    #The highest is the first so if the key is already in the dictionary skip so no others are added for that locus tag
    if locus in locus_dict.keys():
        pass
    else:
        locus_dict[locus] = BGC_id

#file_list = list 
for key, value in locus_dict.items():
    #print(key)
    for line in lines:
        if str(key) in line:
            print(key)
            line = line + value
#print(lines)

f.close()

In [38]:
#can add a for loop to find the highest scoring MiBig identifier for the locus tag 
f = open("../OUTPUTS/antismash_finals/T_712_11/mg.assembly.merged.length_filtered/knownclusterblastoutput.txt", 'r')

# if attributess.__contains__('locus_tag'):
#     locus_tag = re.search(';locus_tag=(.+?);', attributess)
#     locus_name = locus_tag.group(1)

#dictionary to store the MiBig identifier with the corresponding locus_tag 
locus_dict = {}    

for Line in f:
    columns = Line.split('|')
    locus = columns[4]
    BGC_id = columns[5].split('\t')[1]
    #print(locus_name + locus)
    if locus in locus_dict.keys():
        pass
    else:
        locus_dict[locus] = BGC_id


    

    

{'JJJEIFBD_2894428': 'BGC0001751'}
{'JJJEIFBD_2894428': 'BGC0001751', 'JJJEIFBD_2894431': 'BGC0002007'}
{'JJJEIFBD_2894428': 'BGC0001751', 'JJJEIFBD_2894431': 'BGC0002007', 'JJJEIFBD_2894432': 'BGC0000268'}
{'JJJEIFBD_2894428': 'BGC0001751', 'JJJEIFBD_2894431': 'BGC0002007', 'JJJEIFBD_2894432': 'BGC0000268', 'JJJEIFBD_2894434': 'BGC0001061'}
{'JJJEIFBD_2894428': 'BGC0001751', 'JJJEIFBD_2894431': 'BGC0002007', 'JJJEIFBD_2894432': 'BGC0000268', 'JJJEIFBD_2894434': 'BGC0001061', 'JJJEIFBD_2894435': 'BGC0000817'}
{'JJJEIFBD_2894428': 'BGC0001751', 'JJJEIFBD_2894431': 'BGC0002007', 'JJJEIFBD_2894432': 'BGC0000268', 'JJJEIFBD_2894434': 'BGC0001061', 'JJJEIFBD_2894435': 'BGC0000817', 'JJJEIFBD_2894436': 'BGC0001069'}
{'JJJEIFBD_2894428': 'BGC0001751', 'JJJEIFBD_2894431': 'BGC0002007', 'JJJEIFBD_2894432': 'BGC0000268', 'JJJEIFBD_2894434': 'BGC0001061', 'JJJEIFBD_2894435': 'BGC0000817', 'JJJEIFBD_2894436': 'BGC0001069', 'JJJEIFBD_2894437': 'BGC0001520'}
{'JJJEIFBD_2894428': 'BGC0001751', 'JJJEI

In [6]:

for i in range(len(genome_record.features)):
    for key in genome_record.features[i].qualifiers:
        print(genome_record.features[i].qualifiers[key])
    #if genome_record.features[i].type == 'CDS':
    #    print(genome_record.features[i])
print(genome_record.features[0:5])

['rule-based-clusters']
['RiPP']
['True']
['[5459:10452]']
['20000']
['((LANC_like and (Lant_dehydr_N or Lant_dehydr_C)) and not (YcaO or TIGR03882))']
['10000']
['lanthipeptide-class-i']
['1']
['antismash']
['rule-based-clusters']
['antismash']
['20000']
['((LANC_like and (Lant_dehydr_N or Lant_dehydr_C)) and not (YcaO or TIGR03882))']
['10000']
['lanthipeptide-class-i']
['1']
['1']
['True']
['((LANC_like and (Lant_dehydr_N or Lant_dehydr_C)) and not (YcaO or TIGR03882))']
['single']
['lanthipeptide-class-i']
['1']
['antismash']
['1']
['True']
['lanthipeptide-class-i']
['1']
['((LANC_like and (Lant_dehydr_N or Lant_dehydr_C)) and not (YcaO or TIGR03882))']
['antismash']
['EMEGMFMJ_342191']
['EMEGMFMJ_342191']
['ab initio prediction:Prodigal_v2.6.3']
['EMEGMFMJ_342191']
['00']
['0']
['unannotated protein']
['Prodigal_v2.6.3']
['11']
['MEPLPGIRIRRLRLVGMERSYDVDFTAEQRVRNLSVIAGAFSSGKTAVLEFIAYGLGAKRHPRHQEVLRRVRSCLMEVELSGEPHVIERSVGEPSKVAFVRRGTLQSRPTSRAESRTIDPPGAPESLSSLLLSHCKLEGVQLREAPTNSESRT

In [7]:
print(str(genome_record.features[0].qualifiers['tool'][0]) + ' ' + str(genome_record.features[0].type) + '\t' + str(genome_record.features[0].location.start) + '\t' + str(genome_record.features[0].location.end))

antismash protocluster	0	13115


In [8]:
#print(genome_record.features[0].qualifiers.__contains__('aStool'))
print(str(genome_record.features[0].location)[-2])

+


In [9]:
print(dir(genome_record.features[0].location))

['__add__', '__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_end', '_flip', '_get_strand', '_set_strand', '_shift', '_start', '_strand', 'end', 'extract', 'nofuzzy_end', 'nofuzzy_start', 'parts', 'ref', 'ref_db', 'start', 'strand']


In [10]:
attributess = ''
#qualif_dict = genome_record.features[12].qualifiers
for i in genome_record.features[12].qualifiers:
#for i in qualif_dict:
    #print(i, qualif_dict[i])
    
    if i ==  'ID' or i == 'gene' or i == 'translation' or i == 'partial' or i == 'domain_id':
        pass
    else:
        temp = str(i) + '=' + str(genome_record.features[12].qualifiers[i][0]) + ';'
        #temp = str(i) + '=' + str(qualif_dict[i][0]) + ';' 
        attributess = attributess + str(temp)
print(attributess)

gene_functions=biosynthetic (rule-based-clusters) lanthipeptide-class-i: Lant_dehydr_C;gene_kind=biosynthetic;inference=ab initio prediction:Prodigal_v2.6.3;locus_tag=EMEGMFMJ_342197;note=smCOG tree PNG image: smcogs/EMEGMFMJ_342197.png;phase=0;product=unannotated protein;sec_met_domain=Lant_dehydr_N (E-value: 4e-113, bitscore: 368.3, seeds: 73, tool: rule-based-clusters);source=Prodigal_v2.6.3;transl_table=11;


In [11]:
#print(genome_record)
#print(genome_record.features)
#print(dir(genome_record))
print(genome_record.features[10])
print(type(genome_record.features))
print(dir(genome_record.features))
print(dir(genome_record))

type: CDS
location: [6310:7552](-)
qualifiers:
    Key: ID, Value: ['EMEGMFMJ_342196']
    Key: gene, Value: ['EMEGMFMJ_342196']
    Key: gene_functions, Value: ['biosynthetic (rule-based-clusters) lanthipeptide-class-i: LANC_like', 'biosynthetic-additional (smcogs) SMCOG1140:Lanthionine synthetase C family protein (Score: 365.1; E-value: 7.1e-111)']
    Key: gene_kind, Value: ['biosynthetic']
    Key: inference, Value: ['ab initio prediction:Prodigal_v2.6.3']
    Key: locus_tag, Value: ['EMEGMFMJ_342196']
    Key: note, Value: ['smCOG tree PNG image: smcogs/EMEGMFMJ_342196.png']
    Key: partial, Value: ['00']
    Key: phase, Value: ['0']
    Key: product, Value: ['unannotated protein']
    Key: sec_met_domain, Value: ['LANC_like (E-value: 4.4e-44, bitscore: 140.3, seeds: 47, tool: rule-based-clusters)']
    Key: source, Value: ['Prodigal_v2.6.3']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ['VTTALTSAVLSTTDKIAKALRDPSSAWPNGRPAGGRAWPQSLAGGACGIALLHIERARTGHGDWDTAHTWL

In [12]:
print(genome_record.description + '\t' + 'str')

test_contig_245285	str


In [13]:

for i in genome_record.features:
    #print(i.type)
    print(i.type) #print(i.location)

protocluster
proto_core
cand_cluster
region
CDS
CDS
CDS
CDS
CDS
PFAM_domain
CDS
PFAM_domain
CDS
PFAM_domain
PFAM_domain
PFAM_domain
misc_feature
CDS
CDS_motif
CDS_motif
CDS
PFAM_domain
CDS
CDS
CDS


In [21]:
#write gff outfile with all the types entrees for antismash annotation from gbk file 
with open(r'./mg.all.gff', 'w') as out:
    #for line in lines: 
    #    out.write('\n' % line)
    out.write('\n'.join(lines))

#write gff outfile for CDS regions which have actual 
with open(r'./mg.CDS.counts.gff', 'w') as CDS_out:
    for line in lines:
        if line.__contains__('gene_functions'):
            CDS_out.write(line + '\n')

with open(r'./mg.Pfam.counts.gff', 'w') as Pfam_out:
    for line in lines:
        if line.__contains__('PFAM_domain'):
            Pfam_out.write(line + '\n')

In [15]:
print(dir(str))

['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isascii', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'partition', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill']


In [22]:
def make_lines(file_path):
    genome_record = SeqIO.read(file_path, 'genbank')
    
    lines = []
    line = ''
    for i in range(len(genome_record.features)):
        #check if the type of region/feature is a coding sequence/annotated region with the 'phase' qualifier/key, to also include along the line
        if genome_record.features[i].qualifiers.__contains__('phase'):
            #check if annotation is from a 'tool' a 'aStool' or a 'source' to include in the second column 
            if genome_record.features[i].qualifiers.__contains__('tool'):
                string = str(genome_record.description) + '\t' + str(genome_record.features[i].qualifiers['tool'][0]) + ' ' + str(genome_record.features[i].type) + '\t' + str(genome_record.features[i].location.start) + '\t' + str(genome_record.features[i].location.end) + '\t' + '.' + '\t' + str(genome_record.features[i].location)[-2] + '\t' + str(genome_record.features[i].qualifiers['phase'][0]) + '\t'
            else:
                if genome_record.features[i].qualifiers.__contains__('aStool'):
                    string = str(genome_record.description) + '\t' + str(genome_record.features[i].qualifiers['aStool'][0]) + ' ' + str(genome_record.features[i].type) + '\t' + str(genome_record.features[i].location.start) + '\t' + str(genome_record.features[i].location.end) + '\t' + '.' + '\t' + str(genome_record.features[i].location)[-2] + '\t' + str(genome_record.features[i].qualifiers['phase'][0]) + '\t'
                else:
                    string = str(genome_record.description) + '\t' + str(genome_record.features[i].qualifiers['source'][0]) + ' ' + str(genome_record.features[i].type) + '\t' + str(genome_record.features[i].location.start) + '\t' + str(genome_record.features[i].location.end) + '\t' + '.' + '\t' + str(genome_record.features[i].location)[-2] + '\t' + str(genome_record.features[i].qualifiers['phase'][0]) + '\t'
        else:
            if genome_record.features[i].qualifiers.__contains__('tool'):
                string = str(genome_record.description) + '\t' + str(genome_record.features[i].qualifiers['tool'][0]) + ' ' + str(genome_record.features[i].type) + '\t' + str(genome_record.features[i].location.start) + '\t' + str(genome_record.features[i].location.end) + '\t' + '.' + '\t' + str(genome_record.features[i].location)[-2] + '\t' 
            else:
                if genome_record.features[i].qualifiers.__contains__('aStool'):
                    string = str(genome_record.description) + '\t' + str(genome_record.features[i].qualifiers['aStool'][0]) + ' ' + str(genome_record.features[i].type) + '\t' + str(genome_record.features[i].location.start) + '\t' + str(genome_record.features[i].location.end) + '\t' + '.' + '\t' + str(genome_record.features[i].location)[-2] + '\t'
                else:
                    string = str(genome_record.description) + '\t' + str(genome_record.features[i].qualifiers['source'][0]) + ' ' + str(genome_record.features[i].type) + '\t' + str(genome_record.features[i].location.start) + '\t' + str(genome_record.features[i].location.end) + '\t' + '.' + '\t' + str(genome_record.features[i].location)[-2] + '\t'
        
        attributess = ''
        for key in genome_record.features[i].qualifiers:
        #print(i, qualif_dict[i])
            if key ==  'ID' or key == 'gene' or key == 'translation' or key == 'partial' or key == 'domain_id':
                pass
            else:
                if len(genome_record.features[i].qualifiers[key]) > 1:
                    temp = str(key) + '=' + str(genome_record.features[i].qualifiers[key]) + ';' 
                    attributess = attributess + str(temp)
                else:
                #print(len(genome_record.features[i].qualifiers[key]))
                    temp = str(key) + '=' + str(genome_record.features[i].qualifiers[key][0]) + ';' 
                    attributess = attributess + str(temp)
        #attributess 
        line = line + string + attributess
        lines.append(str(line))
        line = ''
    #return list of all the lines 
    return lines 


In [26]:
import glob 
filenames = glob.glob('../OUTPUTS/IMP3_assembly/T_722_5/antismash_output' + "/test_contig_*.gbk") 

#loop over all the gbk files in the list and apply the above function to all
file_list = []
#for each file(by calling the name) in the list of file names
for file in filenames:
    #
    tmp_file = make_lines(file)
    file_list = file_list + tmp_file

In [28]:
#write gff outfile with all the types entrees for antismash annotation from gbk file 
with open(r'./mg.all.gff', 'w') as out:
    #for line in lines: 
    #    out.write('\n' % line)
    out.write('\n'.join(file_list))

#write gff outfile for CDS regions which have actual annotated locus tags with identified gene_functions
with open(r'./mg.CDS.counts.gff', 'w') as CDS_out:
    for line in file_list:
        if line.__contains__('gene_functions'):
            CDS_out.write(line + '\n')

#write gff outfile for found regions from Pfam database 
with open(r'./mg.Pfam.counts.gff', 'w') as Pfam_out:
    for line in file_list:
        if line.__contains__('PFAM_domain'):
            Pfam_out.write(line + '\n')